In [328]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Важная настройка для корректной настройки pipeline!
import sklearn
sklearn.set_config(transform_output="pandas")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, OrdinalEncoder, TargetEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, KFold
import category_encoders as ce
# for model learning
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

#models
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import FunctionTransformer
# Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error 


# tunning hyperparamters model
import optuna

In [329]:
path = "D:/phase_1_repeat/project_2_house/train.csv"

In [330]:
df = pd.read_csv(path)
df


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [331]:
df_add = pd.read_csv('D:/phase_1_repeat/project_2_house/Data_claster.csv')
df_add = df_add[['cluster_kmeans', 'cluster_agglo',	'house_cluster']]
df_add

,cluster_kmeans,cluster_agglo,house_cluster
0,2,1,2
1,3,0,2
2,2,1,2
3,0,2,0
4,1,3,1
...,...,...,...
1455,2,1,2
1456,2,2,2
1457,0,2,0
1458,3,0,3


In [332]:
# df[['GarageFinish', 'Street', 'FireplaceQu', 'MasVnrType','BsmtFinType1', 'BsmtExposure', 'CentralAir','GarageType', 'PoolQC','Fence','MiscFeature' ]] = df[['GarageFinish', 'Street', 'FireplaceQu', 'MasVnrType','BsmtFinType1', 'BsmtExposure', 'CentralAir','GarageType', 'PoolQC','Fence','MiscFeature' ]].fillna('no')

In [333]:
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Пропущено_значений': df.isnull().sum(),
    'Процент_пропусков': missing_percent
})

 #процента пропусков
missing_df = missing_df.sort_values('Процент_пропусков', ascending=False)

print("Таблица пропусков (отсортированная):")
print(missing_df.iloc[: 20])

Таблица пропусков (отсортированная):
              Пропущено_значений  Процент_пропусков
PoolQC                      1453          99.520548
MiscFeature                 1406          96.301370
Alley                       1369          93.767123
Fence                       1179          80.753425
MasVnrType                   872          59.726027
FireplaceQu                  690          47.260274
LotFrontage                  259          17.739726
GarageQual                    81           5.547945
GarageFinish                  81           5.547945
GarageType                    81           5.547945
GarageYrBlt                   81           5.547945
GarageCond                    81           5.547945
BsmtFinType2                  38           2.602740
BsmtExposure                  38           2.602740
BsmtCond                      37           2.534247
BsmtQual                      37           2.534247
BsmtFinType1                  37           2.534247
MasVnrArea                 

In [334]:
df_copy = df.copy()
# categorical_features_cor = df.select_dtypes(include=['object', 'category']).columns.tolist()

# le = LabelEncoder()

# # Примените к каждому категориальному признаку
# for feature in categorical_features_cor:
#     df_copy[feature] = le.fit_transform(df_copy[feature])
# categorical_df = df_copy[categorical_features_cor]
# plt.figure(figsize=(20, 18))
# sns.heatmap(categorical_df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
# plt.title('Корреляция числовых признаков')
# plt.show()


# numerical_features_cor = df_copy.select_dtypes(include=['number']).columns.tolist()
# numerical_df = df_copy[numerical_features_cor]
# plt.figure(figsize=(20, 18))
# sns.heatmap(numerical_df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
# plt.title('Корреляция числовых признаков')
# plt.show()

In [335]:
# percent = (df_copy.isnull().sum() / len(df_copy)) * 100
# df_new = pd.DataFrame({
#     'Пропущено_значений': df_copy.isnull().sum(),
#     'Процент_пропусков': percent
# })

#  #процента пропусков
# df_new= df_new.sort_values(by='Процент_пропусков', ascending=False, axis=0)

# print("Таблица пропусков (отсортированная):")
# print(df_new.iloc[: 20])

In [336]:
X,y = df.drop("SalePrice", axis=1),df["SalePrice"]

In [337]:
# y =df_copy["SalePrice"]
# plt.figure()
# plt.scatter(range(len(y)), y)
# plt.xlabel("Индекс наблюдения")
# plt.ylabel("Значение целевой переменной")
# plt.title("Scatter plot целевой переменной HasDetections")
# plt.grid(True)
# plt.show()

In [338]:
X_train, X_valid, y_train, y_valid =  train_test_split(X,y, test_size=0.2, random_state=42)

In [339]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [340]:
numerical_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
# drop_columns = ["PoolQC","MiscFeature","Alley", "Fence", "MasVnrType","FireplaceQu", "LotFrontage","GarageYrBlt","TotRmsAbvGrd","GarageCars","TotalBsmtSF", 'Exterior2nd', 'LotConfig', 'GarageCond', 'Fence', 'Id', 'GarageArea', 'BsmtCond', 'BldgType', 'KitchenQual', 'ExterQual', 'Alley', 'SaleCondition', 'PavedDrive', 'GarageQual', 'Condition2', 'YearRemodAdd', 'LotFrontage', 'YearBuilt', 'RoofMatl', 'BsmtExposure', 'BsmtUnfSF', 'OverallQual', 'MiscFeature', 'LotShape', 'Condition1', 'Neighborhood', 'BsmtFinType2', 'MasVnrType', 'Heating', 'Foundation', 'LandSlope', 'GarageCars', 'GarageType', 'BsmtFinSF2', 'YrSold', 'MasVnrArea', 'MSZoning', 'MSSubClass', 'Street', 'HeatingQC', 'TotalBsmtSF']
drop_columns = ['Id', '1stFlrSF', '2ndFlrSF', 'ExterQual', 'BsmtFinSF1', 'GarageYrBlt', 'TotRmsAbvGrd', 'GarageCars', 'PoolQC', 'MasVnrArea', 'TotRmsAbvGrd', 'YearRemodAdd', 'FullBath', '3SsnPorch', 'LotShape', 'FireplaceQu', 'HalfBath', 'MasVnrType', 'BsmtFinType2', 'PavedDrive', 'BsmtCond', 'Foundation', 'KitchenAbvGr', 'RoofStyle', 'HouseStyle', 'GarageQual', 'RoofMatl', 'Electrical', 'BldgType']
numerical_features = [col for col in numerical_features if col not in drop_columns]
categorical_features = [col for col in categorical_features if col not in drop_columns]

In [341]:
# log_columns = []
# df_for_columns = X_train.describe()
# for column in df_for_columns.columns:
#     col_min = df_for_columns.loc['min', column]
#     col_max = df_for_columns.loc['max', column]
#     if col_min > 1 and col_max > 100:
#         log_columns.append(column)
# log_columns

In [342]:
# df['SalePrice_log'] = np.log1p(df['SalePrice'])


# # 2. Skewed числовые колонки

# skew_threshold = 0.75  # порог скоса

# for col in log_columns:
#     if col not in ['Id', 'SalePrice', 'SalePrice_log']:
#         # Проверяем на положительные значения
#         if (df[col] >= 0).all() and df[col].nunique() > 10:
#             skewness = df[col].skew()
#             if abs(skewness) > skew_threshold:
#                 # Создаем логарифмированную версию
#                 df[f'{col}_log'] = np.log1p(df[col])
#                 print(f"✅ {col}: skew={skewness:.2f} -> логарифмирована")

# print(f"\nВсего числовых колонок: {len(log_columns)}")
# print(f"Логарифмированных колонок: {len([c for c in df.columns if '_log' in c])}")

In [343]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [344]:
log_transformer = FunctionTransformer(np.log1p, validate=False)


In [345]:
# def combine_feature_total1st2ndfloor(X):
#         # X — это DataFrame или array
#     # Добавляем новую колонку: income + price
#     result = X.copy()
#     if isinstance(X, np.ndarray):
#         # Если X — numpy array, нужно знать индексы
#         # Предположим, что income — 0-й столбец, price — 1-й
#         result = np.column_stack([
#             X,
#             X[:, 0] + X[:, 1]  # income + price
#         ])
#     else:
#         result['Totalfloors'] = X['1stFlrSF'] + X['2ndFlrSF']
#     return result

# # def combine_feature_TotalBath(X):
# #         # X — это DataFrame или array
# #     # Добавляем новую колонку: income + price
# #     result = X.copy()
# #     if isinstance(X, np.ndarray):
# #         # Если X — numpy array, нужно знать индексы
# #         # Предположим, что income — 0-й столбец, price — 1-й
# #         result = np.column_stack([
# #             X,
# #             X[:, 0] + X[:, 1]  # income + price
# #         ])
# #     else:
# #         result['TotalBath'] = X['FullBath'] + X['BsmtFullBath'] + 0.5*X['BsmtHalfBath'] + 0.5* X['HalfBath']
# #     return result

# # def combine_feature_PorchDeckSF(X):
# #         # X — это DataFrame или array
# #     # Добавляем новую колонку: income + price
# #     result = X.copy()
# #     if isinstance(X, np.ndarray):
# #         # Если X — numpy array, нужно знать индексы
# #         # Предположим, что income — 0-й столбец, price — 1-й
# #         result = np.column_stack([
# #             X,
# #             X[:, 0] + X[:, 1]  # income + price
# #         ])
# #     else:
# #         result['PorchDeckSF'] = X['OpenPorchSF'] + X['EnclosedPorch'] + X['3SsnPorch'] + X['ScreenPorch'] + X['WoodDeckSF']
# #     return result



In [346]:

imputer = ColumnTransformer(
    transformers= [
        ("numerical_features", SimpleImputer(strategy ="median"), numerical_features),
        ("categorical_features", SimpleImputer(strategy="most_frequent"), categorical_features)
        ],
    verbose_feature_names_out=False,
    remainder= 'passthrough'
    
)

In [347]:

imputer_drop = ColumnTransformer(
    transformers=[("drop", "drop", drop_columns)],
    verbose_feature_names_out=False,
    remainder= 'passthrough'

)

In [348]:
# feature_columns = df_copy.columns.drop('SalePrice', errors='ignore')  # или просто data.columns, если все — признаки

# # Словарь: ключ — количество уникальных значений, значение — список имён столбцов
# columns_by_nunique = {}

# for col in feature_columns:
#     n_unique = df_copy[col].nunique(dropna=True)  # dropna=True — по умолчанию, но явно указываем
#     if n_unique not in columns_by_nunique:
#         columns_by_nunique[n_unique] = []
#     columns_by_nunique[n_unique].append(col)

# # Теперь можно создать отдельные списки, например:
# list_3 = columns_by_nunique.get(3, [])
# list_4 = columns_by_nunique.get(4, [])
# list_5 = columns_by_nunique.get(5, [])

In [349]:
encoder_and_scaler =ColumnTransformer(
transformers=[

        ('encoder', ce.CatBoostEncoder(),categorical_features),
        ('scaler', StandardScaler(), numerical_features)
    ],
    verbose_feature_names_out=False,
    remainder= 'passthrough'
)

In [350]:
# log =ColumnTransformer(transformers=[
#         ('log', FunctionTransformer(np.log1p, validate=False), numerical_features)],
#     verbose_feature_names_out=False,
#     remainder= 'passthrough'
# )

In [351]:
preprocessor = Pipeline([
     ('imputer_drop', imputer_drop), 
    ("imputer", imputer),

    # ('log', log),
    ("encoder_and_scaler", encoder_and_scaler)]
)

In [352]:
preprocessor

,steps,"[('imputer_drop', ...), ('imputer', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('drop', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [353]:
print(categorical_features)

['MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'Exterior1st', 'Exterior2nd', 'ExterCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageCond', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [354]:
# X_train_prep = preprocessor.fit_transform(X_train, y_train)
# X_valid_prep = preprocessor.fit_transform(X_valid, y_valid)

In [355]:
# models = {
#     "linereg": LinearRegression(),
#     "Ridge": Ridge(),
#     "Lasso": Lasso(),
#     "RandomForest": RandomForestRegressor(),
#     "GradientBoosting": GradientBoostingRegressor(),
#     "KNN": KNeighborsRegressor(),
#     "XGB": XGBRegressor(),
#     "LGBM": LGBMRegressor()}
# result = {}

# for name, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_valid)
    
#     r2 = r2_score(y_valid, y_pred)
#     rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
#     mae = mean_absolute_error(y_valid, y_pred)
#     rmlse = np.sqrt(mean_squared_log_error(y_valid, y_pred))
    
    
#     result[name]={
#         "model": model,
#         "r2": r2,
#         "rmse": rmse,
#         "mae": mae,
#         "rmlse": rmlse        
#     }
#     print(f" {name} / {r2:.4f} / {rmse:.4f} / {mae:.4f} / {rmlse:.4f} ")
    
# result_df = pd.DataFrame([{
#     "model": name,
#     "r2": metrics["r2"],
#     "rmse": metrics["rmse"],
#     "mae": metrics["mae"],
#     "rmlse": metrics["rmlse"],
# }
# for name, metrics in result.items()
                              
# ]) 

# print(result_df.to_string(index=False))

In [356]:
# models = {
#     "linereg": LinearRegression(),
#     "Ridge": Ridge(),
#     "Lasso": Lasso(),
#     "RandomForest": RandomForestRegressor(),
#     "GradientBoosting": GradientBoostingRegressor(
#         n_estimators = 253, max_depth = 5, 
#         learning_rate = 0.024850394952947306, 
#         subsample = 0.8433434535775448
#     ),
#     "KNN": KNeighborsRegressor(),
#     "XGB": XGBRegressor(
#         n_estimators = 506, max_depth = 5, 
#         learning_rate = 0.024850394952947306, 
#         subsample = 0.8433434535775448
#     ),
#     "LGBM": LGBMRegressor(
#         n_estimators= 740, max_depth= 3, learning_rate= 0.05781374893206464, num_leaves= 29, subsample= 0.8708584820389839, colsample_bytree= 0.8804619556540367, min_child_samples = 19, reg_alpha = 0.0014575077384129489, reg_lambda= 2.0665218739809577e-05, min_samples_split= 36, min_samples_leaf= 12, max_features= 'auto'
#     )
    
# }

# result = {}

# for name, model in models.items():
#     # Обучаем на ПОДГОТОВЛЕННЫХ данных
#     model.fit(X_train_prep, y_train)
#     y_pred = model.predict(X_valid_prep)
    
#     r2 = r2_score(y_valid, y_pred)
#     rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
#     mae = mean_absolute_error(y_valid, y_pred)
    
#     # RMSLE только если все значения > 0
#     try:
#         rmlse = np.sqrt(mean_squared_log_error(y_valid, y_pred))
#     except ValueError:
#         rmlse = float('nan')
    
#     result[name] = {
#         "r2": r2,
#         "rmse": rmse,
#         "mae": mae,
#         "rmlse": rmlse
#     }
#     print(f"{name} / R2: {r2:.4f} / RMSE: {rmse:.4f} / MAE: {mae:.4f} / RMSLE: {rmlse:.4f}")

In [357]:
# preprocessor.fit(X_train) 
# preprocessor.fit(X_valid)
preprocessor.fit(X_train, y_train)  # ← важно!
X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)

In [358]:
print("NaN в X_train:", X_train.isnull().sum().sum())
print("NaN в y_train:", y_train.isnull().sum())

NaN в X_train: 0
NaN в y_train: 0


In [359]:
missing_percent = (X_train.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Пропущено_значений': X_train.isnull().sum(),
    'Процент_пропусков': missing_percent
})

 #процента пропусков
missing_df = missing_df.sort_values('Процент_пропусков', ascending=False)

print("Таблица пропусков (отсортированная):")
print(missing_df.iloc[: 20])

Таблица пропусков (отсортированная):
              Пропущено_значений  Процент_пропусков
MSZoning                       0                0.0
Street                         0                0.0
Alley                          0                0.0
LandContour                    0                0.0
Utilities                      0                0.0
LotConfig                      0                0.0
LandSlope                      0                0.0
Neighborhood                   0                0.0
Condition1                     0                0.0
Condition2                     0                0.0
Exterior1st                    0                0.0
Exterior2nd                    0                0.0
ExterCond                      0                0.0
BsmtQual                       0                0.0
BsmtExposure                   0                0.0
BsmtFinType1                   0                0.0
Heating                        0                0.0
HeatingQC                  

In [360]:
# y_train_log = np.log1p(y_train)

In [361]:

model_gbr =GradientBoostingRegressor(
        n_estimators = 600, max_depth = 4, 
        learning_rate = 0.024850394952947306, 
        subsample = 0.8433434535775448
    )

model_gbr.fit(X_train, y_train)

model_lgbm = LGBMRegressor(
    n_estimators=740,
    max_depth=3,
    learning_rate=0.05781374893206464,
    num_leaves=29,
    bagging_fraction=0.8708584820389839,
    feature_fraction=0.8804619556540367,
    min_child_samples=19,
    min_data_in_leaf=12,
    reg_alpha=0.0014575077384129489,
    reg_lambda=2.0665218739809577e-05,
    random_state=42
    )
# model_lgbm.fit(X_train, y_train)
model_cat = CatBoostRegressor()

# model = VotingRegressor(

#     [('GradientBoostingRegressor', model_gbr),
#     ('CatBoostRegressor', model_cat),
#     ('LGBMRegressor', model_lgbm)
#     ],
#     weights = [2.5, 2.0, 0.6]  

# )

# model.fit(X_train, y_train)


In [362]:
# После предсказания в лог-шкале
y_pred = model_gbr.predict(X_valid)

# # Обрезаем экстремальные значения
# y_pred_log_safe = np.clip(y_pred_log, a_min=None, a_max=700)

# # Возвращаем в исходную шкалу
# y_pred = np.expm1(y_pred_log_safe)

r2_scores = r2_score(y_valid, y_pred)
r2_scores

0.9093483067524288

In [363]:
X_test =  pd.read_csv("D:/phase_1_repeat/project_2_house/test.csv")
X_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [364]:
X_test = preprocessor.transform(X_test)
y_pred_kaggle = model_gbr.predict(X_test)
# y_pred_kaggle = np.expm1(y_pred_kaggle_log) 
y_pred_kaggle

array([131711.22518426, 153546.17860199, 183671.04219608, ...,
       170216.73211356, 125307.49322861, 191756.89987007], shape=(1459,))

In [365]:
X_test.head()

,MSZoning,Street,Alley,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,134585.971372,181511.598748,181765.860002,180623.798624,181479.163135,178463.327511,180275.169109,147595.200780,142497.978238,181489.643819,...,-0.958592,1.202465,0.339836,-0.714352,-0.351921,1.872497,-0.070993,-0.092740,-0.133417,1.650065
1,190330.288153,181511.598748,181765.860002,180623.798624,181479.163135,176722.434874,180275.169109,147595.200780,184632.718947,181489.643819,...,-0.958592,-0.778531,2.291539,-0.195649,-0.351921,-0.275838,-0.070993,22.519298,-0.133417,1.650065
2,190330.288153,181511.598748,181765.860002,180623.798624,181479.163135,178463.327511,180275.169109,196311.871848,184632.718947,181489.643819,...,0.592150,0.027137,0.895261,-0.224465,-0.351921,-0.275838,-0.070993,-0.092740,-1.257196,1.650065
3,190330.288153,181511.598748,181765.860002,180623.798624,181479.163135,178463.327511,180275.169109,196311.871848,184632.718947,181489.643819,...,0.592150,-0.029734,2.036969,-0.195649,-0.351921,-0.275838,-0.070993,-0.092740,-0.133417,1.650065
4,190330.288153,181511.598748,181765.860002,243320.626165,181479.163135,178463.327511,180275.169109,296835.978188,184632.718947,181489.643819,...,-0.958592,0.140878,-0.740157,0.467139,-0.351921,2.302164,-0.070993,-0.092740,-2.006382,1.650065


In [366]:
X_test_2 = pd.read_csv("D:/phase_1_repeat/project_2_house/test.csv")

In [367]:
submission = pd.DataFrame({
    'Id': X_test_2['Id'],           # ← обязательно из test.csv!
    'SalePrice': y_pred_kaggle
})
submission.set_index('Id', inplace=True)
# submission = submission.reset_index() 
submission.to_csv('submission.csv', index=True)

In [368]:
submission

,SalePrice
Id,
1461,131711.225184
1462,153546.178602
1463,183671.042196
1464,198250.527679
1465,188438.638750
...,...
2915,88018.896649
2916,78152.568201
2917,170216.732114


In [369]:
X_train_del = preprocessor.transform(X_train)

# Получаем имена преобразованных признаков
feature_names = preprocessor.get_feature_names_out()

# Теперь создаём Series
check = pd.Series(
    model_gbr.feature_importances_,
    index=feature_names
).sort_values(key=lambda x: abs(x), ascending=False)

In [370]:
check.iloc[:25]

OverallQual      0.479728
GrLivArea        0.147530
Neighborhood     0.107637
TotalBsmtSF      0.066420
LotArea          0.024255
GarageArea       0.020679
BsmtQual         0.020155
KitchenQual      0.017410
LotFrontage      0.011808
YearBuilt        0.010535
GarageFinish     0.008232
BsmtUnfSF        0.008072
BsmtFullBath     0.007093
OverallCond      0.006897
Fireplaces       0.006219
PoolArea         0.004662
SaleCondition    0.004577
BsmtExposure     0.004464
ScreenPorch      0.003845
CentralAir       0.003765
LandContour      0.003535
SaleType         0.003504
BsmtFinType1     0.003436
OpenPorchSF      0.002932
WoodDeckSF       0.002849
dtype: float64

In [371]:
check.iloc[25:50]

Exterior2nd      0.002094
MoSold           0.002035
MSSubClass       0.001779
LotConfig        0.001624
BedroomAbvGr     0.001496
Condition1       0.001449
GarageType       0.001413
MSZoning         0.001311
Exterior1st      0.001213
YrSold           0.000937
EnclosedPorch    0.000842
Functional       0.000831
HeatingQC        0.000686
LowQualFinSF     0.000520
BsmtFinSF2       0.000411
Alley            0.000390
ExterCond        0.000370
Fence            0.000102
BsmtHalfBath     0.000077
GarageCond       0.000065
Heating          0.000042
LandSlope        0.000039
Condition2       0.000015
MiscFeature      0.000008
Utilities        0.000007
dtype: float64

In [372]:
check.iloc[50:75]

MiscVal    0.000005
Street     0.000000
dtype: float64

In [373]:
feature_names = check.iloc[10:].index.tolist()
print(feature_names)

['GarageFinish', 'BsmtUnfSF', 'BsmtFullBath', 'OverallCond', 'Fireplaces', 'PoolArea', 'SaleCondition', 'BsmtExposure', 'ScreenPorch', 'CentralAir', 'LandContour', 'SaleType', 'BsmtFinType1', 'OpenPorchSF', 'WoodDeckSF', 'Exterior2nd', 'MoSold', 'MSSubClass', 'LotConfig', 'BedroomAbvGr', 'Condition1', 'GarageType', 'MSZoning', 'Exterior1st', 'YrSold', 'EnclosedPorch', 'Functional', 'HeatingQC', 'LowQualFinSF', 'BsmtFinSF2', 'Alley', 'ExterCond', 'Fence', 'BsmtHalfBath', 'GarageCond', 'Heating', 'LandSlope', 'Condition2', 'MiscFeature', 'Utilities', 'MiscVal', 'Street']


In [374]:

# model_cat_boost = CatBoostRegressor(
#     # n_estimators= 81, 
#     # max_depth=6, 
#     # learning_rate =  0.09929108743509919,
#     # l2_leaf_reg =  83.86834789304754,
#     # bagging_temperature= 0.9905878484464076,
#     # random_strength = 0.3972742649694947
# )

# model_cat_boost.fit(X_train_optune, y_train)


In [375]:
# X_valid_optune = preprocessor.transform(X_valid)

In [376]:
# y_pred_cat = model_cat_boost.predict(X_valid_optune)

In [377]:
# r2_catboost = r2_score(y_valid, y_pred_cat)
# r2_catboost

In [378]:
# X_train_optune = preprocessor.transform(X_train)
# X_train_optune

In [379]:
# def objective(trial):
#     # Гиперпараметры для регрессии (улучшенные диапазоны)
#     n_estimators = trial.suggest_int('n_estimators', 100, 1000)
#     max_depth = trial.suggest_int('max_depth', 3, 12)          # LightGBM эффективен с меньшей глубиной
#     learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2, log=True)
#     num_leaves = trial.suggest_int('num_leaves', 20, 256)      # max: ~2^max_depth
#     subsample = trial.suggest_float('subsample', 0.6, 1.0)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)
#     min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
#     reg_alpha = trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True)      # L1
#     reg_lambda = trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)    # L2

#     model = LGBMRegressor(
#         n_estimators=trial.suggest_int('n_estimators', 100, 500),
#         max_depth=trial.suggest_int('max_depth', 3, 10),
#         learning_rate=trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
#         subsample=trial.suggest_float('subsample', 0.6, 1.0),
#         min_samples_split = trial.suggest_int('min_samples_split', 2, 100),
#         min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50),
#         max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),

#     )
    
#     # ←←← ИСПРАВЛЕНО: метрика для РЕГРЕССИИ
#     scores = cross_val_score(
#         model, 
#         X_train, 
#         y_train, 
#         cv=5, 
#         scoring='neg_root_mean_squared_error'  # или 'r2'
#     )
    
#     # cross_val_score возвращает отрицательные значения для ошибок
#     return scores.mean()  # это уже -RMSE, но Optuna максимизирует → так правильно

# # Направление: 'maximize', потому что neg_root_mean_squared_error — чем ближе к нулю (меньше ошибка), тем выше значение
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# best_params = study.best_params
# best_score = study.best_value

# print("Лучшие параметры:", best_params)
# print("Лучший CV результат (negative RMSE):", best_score)
# print("Соответствует RMSE ≈", -best_score)